In [1]:
#0
#Read libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#1
#Retrieve data from Wikipedia

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [3]:
#2
#2.1 - Turn data into Pandas dataframe
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood2'])

In [4]:
#2.2 Cleanup Borough column
# Find indexes of rows that have "Not assigned" in Borough column
indexNames = df[(df['Borough'] == "Not assigned")].index

# Drop rows that have "Not assigned" in Borough column
df.drop(indexNames,inplace=True)

# Drop the first row
df.dropna(inplace=True)

In [5]:
#2.3 Collpase data
# Combine multiple rows into one row based on PostalCode and Borough
df=df.groupby(['PostalCode','Borough'])['Neighborhood2'].apply(', '.join).reset_index()

In [6]:
#2.44 Clean Neighbourhood column
# Replace "Not assigned" in Neighborhood column with the value in Borough column
def custom_fx(data):
    if data['Neighborhood2']=='Not assigned':
        var=data['Borough']
    else:
        var=data['Neighborhood2']
    return var

# Apply the function
df['Neighborhood']=df.apply(custom_fx,axis='columns')

# Check that there is no more "Not assigned" in Neighborhood column
print("There are {} rows that have 'Not assigned' in Neighborhood column in the dataframe".format(
    len(df[df['Neighborhood']=='Not assigned'])
)
     )

# Delete Neighborhood2 column
df.drop(columns='Neighborhood2')

There are 0 rows that have 'Not assigned' in Neighborhood column in the dataframe


PostalCode      Borough                                       Neighborhood
0          M1B  Scarborough                                    Malvern / Rouge
1          M1C  Scarborough           Rouge Hill / Port Union / Highland Creek
2          M1E  Scarborough                Guildwood / Morningside / West Hill
3          M1G  Scarborough                                             Woburn
4          M1H  Scarborough                                          Cedarbrae
..         ...          ...                                                ...
98         M9N         York                                             Weston
99         M9P    Etobicoke                                          Westmount
100        M9R    Etobicoke  Kingsview Village / St. Phillips / Martin Grov...
101        M9V    Etobicoke  South Steeles / Silverstone / Humbergate / Jam...
102        M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]

In [7]:
#Export the datafram
df.to_csv(r'C:\Users\thoma\Desktop\Week3\TRT.csv',index=None,header=True)

In [8]:
#3
#Explore dataset
print("The shape of the dataframe is {}. The dataset has {} rows.".format
      (df.shape,df.shape[0]))

The shape of the dataframe is (103, 4). The dataset has 103 rows.
